In [1]:
import os
import sys
try:
    import balsam
except:
    print("""Cannot find balsam, make sure balsam is installed or it is available in Python search paths. 
          See https://balsam.readthedocs.io/en/latest/#installation""")    

try:
    from ipywidgets import interact, interactive
    from ipywidgets import fixed, interact_manual 
    from ipywidgets import Textarea, widgets, Layout, Accordion
    from ipywidgets import VBox, HBox, Box, Text, BoundedIntText
except:
    print('This notebook requires ipywidgest for interactive features. Please install using pip or conda.')

In [2]:
from balsam import django_config
from balsam.django_config.db_index import refresh_db_index

databasepaths = []
databasepaths.extend(refresh_db_index())
print(f'There are {len(databasepaths)} Balsam databases available.')
for i,db in enumerate(databasepaths):
    print(f'{i}: {db}')

@interact(db=[(i,db) for i,db in enumerate(databasepaths)])
def activate_database(db=''):
    """
    Activates Balsam database by setting the BALSAM_DB_PATH environment variable.
    Note: Once BALSAM_DB_PATH is set, you need to restart Jupyter kernel to change it again.
    """
    os.environ["BALSAM_DB_PATH"] = db
    print(f'Selected database: {os.environ["BALSAM_DB_PATH"]}')

There are 11 Balsam databases available.
0: /lus/theta-fs0/projects/datascience/keceli/valence_balsam/balsamdb
1: /lus/theta-fs0/projects/datascience/keceli/balsam/nwchem_demo/db
2: /lus/theta-fs0/projects/datascience/keceli/balsam/simint/simint_db
3: /lus/theta-fs0/projects/datascience/keceli/balsam/balsamdb_general
4: /lus/theta-fs0/projects/datascience/keceli/balsam/jupyter_test
5: /lus/theta-fs0/projects/connectomics_aesp/balsam_database
6: /lus/theta-fs0/projects/datascience/keceli/container/scaling_test/connectomics
7: /lus/theta-fs0/projects/datascience/keceli/nwx/nwx_db
8: /home/keceli/test_balsam
9: /gpfs/mira-home/keceli/test_balsam
10: /home/keceli/test_balsam_theta


A Jupyter Widget

In [3]:
# If balsam server is not running (may happen after a maintanence) you get:
# "OperationalError: could not connect to server: Connection refused"
# This exception is caught here and it tries to restart the server.

from balsam.core.models import ApplicationDefinition as App
from balsam.scripts import postgres_control

try:
    apps = App.objects.all()
    print(f'Found {len(apps)} apps in {os.environ["BALSAM_DB_PATH"]}:')
    for i,app in enumerate(apps):
        print(f'{i}: {app.name}')
except Exception as e:
    if 'could not connect to server' in e:
        print('Exception caught. Could not connect to server.')
        print(f'Trying to restart the Balsam server {os.environ["BALSAM_DB_PATH"]} ...')
        try:
            postgres_control.start_main(os.environ["BALSAM_DB_PATH"])
        except Exception as e:
            print('Exception caught:')
            print(e.with_traceback())         
    else:
        print('Exception caught:')
        print(e.with_traceback())


Found 1 apps in /home/keceli/test_balsam_theta:
0: SayHello


In [4]:
#apps = App.objects.all()
@interact(name='',executable='',checkexe=False,description='',preprocess='',postprocess='',saveapp=False)
def add_app(name, executable, description='', envscript='', preprocess='', postprocess='', checkexe=False,saveapp=False):
    """
    Adds a new app to the balsam database.
    Parameters
    ----------
    name: str, name of the app
    executable: str, path to the executable
    checkexe: boolean, True: check if executable is available
    description: str, info about the app
    preprocess: str, path to the preprocessing script
    postprocess: str, path to the postprocessing script
    saveapp: boolean, True: save app to the database
    """
    from balsam.core.models import ApplicationDefinition as App
    import shutil
    newapp = App()
    if checkexe:
        if shutil.which(executable):        
            print('{} is found'.format(executable))
        else:
            print('{} is not found'.format(executable))
            return newapp
        
    if App.objects.filter(name=name).exists():
        print("An application named {} already exists".format(name))
    else:
        newapp.name        = name
        newapp.executable  = executable
        newapp.description = description
        newapp.envscript   = envscript
        newapp.preprocess  = preprocess
        newapp.postprocess = postprocess
        if saveapp:
            newapp.save()
            print(f'{newapp.name} added to the balsam database.')
    return newapp


A Jupyter Widget

In [5]:
# Not ready, find how to add dictionaries

@interact(name='', workflow='', application=[app.name for app in App.objects.all()], 
          description='', args='', num_nodes=range(1,4394), 
          ranks_per_node=range(1,256),cpu_affinity=['depth','none'],
          data={},environ_vars={''},save=False)
def add_job(name, workflow, application, 
            description='', args='', num_nodes=1, 
            ranks_per_node=1,cpu_affinity='depth',
            data={},environ_vars={''}, save=False):
    from balsam.launcher.dag import BalsamJob
    job                = BalsamJob()
    job.name           = name
    job.workflow       = workflow
    job.application    = application
    job.description    = description
    job.args           = args
    job.num_nodes      = num_nodes
    job.ranks_per_node = ranks_per_node
    job.cpu_affinity   = cpu_affinity
    job.environ_vars   = environ_vars
    job.data           = {}
    if save:
        job.save()
        print(f'{job.name}:{job.job_id} job added to the balsam database.')


A Jupyter Widget

In [6]:
@interact(project='',queue='default',nodes=range(1,4394),
          wall_minutes=range(30,10000),job_mode='mpi',wf_filter='',
          save=False,submit=False)
def submit(project='',queue='',nodes=1,
           wall_minutes=30,job_mode='mpi',wf_filter='',
           save=False,submit=False):
    """
    Submits a job to the queue with the given parameters.
    Parameters
    ----------
    project: str, name of the project to be charged
    queue: str, queue name, can be: 'default', 'debug-cache-quad', or 'debug-flat-quad'
    nodes: int, Number of nodes, can be any integer from 1 to 4096.
    wall_minutes: int, max wall time in minutes
    job_mode: str, Balsam job mode, can be 'mpi', 'serial'
    wf_filter: str, Selects Balsam jobs that matches the given workflow filter.
    """
    from balsam import setup
    setup()
    from balsam.service import service
    from balsam.core import models
    QueuedLaunch = models.QueuedLaunch
    mylaunch = QueuedLaunch()
    mylaunch.project = project
    mylaunch.queue = queue
    mylaunch.nodes = nodes
    mylaunch.wall_minutes = wall_minutes
    mylaunch.job_mode = job_mode
    mylaunch.wf_filter = wf_filter
    mylaunch.prescheduled_only=False
    if save:
        mylaunch.save()
    if submit:
        service.submit_qlaunch(mylaunch, verbose=True)

A Jupyter Widget

In [7]:
@interact(job_id='',show_output=False)
def get_job_info(job_id='',show_output=False):
    """
    Prints verbose job info for a given job id.
    Parameters
    ----------
    job_id: str, Partial or full Balsam job id.
    """
    from balsam.launcher.dag import BalsamJob as Job
    jobs = Job.objects.all().filter(job_id__contains=job_id)
    if len(jobs) == 1:
        thejob = jobs[0]
        print(jobs[0])
        if show_output:
            output = f'{thejob.working_directory}/{thejob.name}.out'
            with open(output) as f:
                out = f.read()
            print(f'Output file {output} content:')
            print(out)
    elif len(jobs) == 0:
        print('No matching jobs')
    else:
        print(f'{len(jobs)} jobs matched, enter full id.')

A Jupyter Widget

In [8]:
from balsam.launcher.dag import BalsamJob as Job
#for job in Job.objects.filter(state='JOB_FINISHED',workflow='wf_test_valence190705').all():
from balsam.core.models import ApplicationDefinition as App
allstates = ['ALL',
             'CREATED',
             'AWAITING_PARENTS',
             'READY',
             'STAGED_IN',
             'PREPROCESSED',
             'RUNNING',
             'RUN_DONE',
             'POSTPROCESSED',
             'JOB_FINISHED',
             'RUN_TIMEOUT',
             'RUN_ERROR',
             'RESTART_READY',
             'FAILED',
             'USER_KILLED']
allworkflows = [wf['workflow'] for wf in Job.objects.order_by().values('workflow').distinct()]
allworkflows.append('ALL')
allapps = [app.name for app in App.objects.all()]
allapps.append('ALL')
@interact(state=allstates,workflow=allworkflows,app=allapps,name='')
def list_jobs(state='ALL',workflow='ALL',app='ALL',name=''):
    jobs = Job.objects.all()
    print(f'Total number of jobs: {len(jobs)}')
    if state != 'ALL':
        jobs = jobs.filter(state=state)
    if workflow != 'ALL':
        jobs = jobs.filter(workflow=workflow)
    if app != 'ALL':
        jobs = jobs.filter(application=app)
    if name:
        jobs = jobs.filter(name__icontains=name)
    print(f'Selected number of jobs: {len(jobs)}')
    if len(jobs) > 0: 
        t = '{:<20}'.format('Name')
        t += ' {:>8}'.format('Nodes')
        t += ' {:>12}'.format('Ranks')
        t += ' {:^8}'.format('ID')
        if state =='JOB_FINISHED':
            t += '{:>12}'.format('Runtime')
        elif state =='ALL':
            t += '{:>15}'.format('State')
        print(t)
        for job in jobs:
            s = '{:<20.15}'.format(job.name)
            s += ' {:>8}'.format(job.num_nodes)
            s += ' {:>12}'.format(job.num_ranks)
            s += '  {:>8}'.format(str(job.job_id).split('-')[0])            

            if state =='JOB_FINISHED':
                s += '{:>12.3f}'.format(job.runtime_seconds)
            elif state =='ALL':
                s += '{:>15}'.format(job.state)
            print(s)

A Jupyter Widget